# Low Resolution Generation to Super Resolution

In this notebook, we first generate a dataset with low resolution and then use a super resolution network to enhance the point cloud to the desired density. The rationale behind this approach is to produce a dataset with high variability due to the limited resolution of the initial network. This simplifies the task for the super resolution network, which is primarily focused on completing and refining the shapes.

Notes
The low resolution dataset may sometimes have missing parts in the subsampled shapes. Fortunately, the super resolution network has been exposed to similar examples during its training, which equips it to some degree to handle shape completion. However, the rarity of such cases might lead to less consistent performance. 
However, the face that the low resolution network may produce objects with missing components indicates a strong adherence to the training distribution it has learned and a high generation variability.

## Set up and Imports

In [3]:
%cd ..

/home/tourloid/Desktop/PhD/Code/SPVD


In [67]:
import os
import torch
import numpy as np
from models.spvd import SPVUnet
from utils.schedulers import DDPMSparseSchedulerGPU
from utils.super_resolution_schedulers import DDPMSparseCompletionSchedulerGPU
from tqdm import tqdm
from test_generation import get_test_loader, evaluate_gen

# Generate the low res dataset

The chair category of shapenet consists of 662 samples.

## Load the generation model

In [7]:
gen_model = SPVUnet(point_channels=3, voxel_size=0.1, num_layers=1, pres=1e-5,
                 down_blocks = [[(64, 128, 192, 256, 384, 384), (True, True, True, True, False), (None, None, None, 8, 8)]], # only one point skip connection during downsampling
                 up_blocks   = [[(384, 384, 256), (True, True), (8, 8), (3, 3)], 
                                [(256, 192, 128, 64), (True, True, False), (None, None, None), (3, 3, 3)]])
gen_model.load_state_dict(torch.load('./checkpoints/chair_low_res_generation.pt')['state_dict'])

<All keys matched successfully>

In [9]:
gen_model = gen_model.cuda().eval()

## Generate the dataset

In [20]:
gen_sched = DDPMSparseSchedulerGPU(sigma='coef_bt')
samples = []
for i in tqdm(range(0, 662, 32)):
    bs = min(662-i, 32)
    samples.extend(gen_sched.sample(gen_model, bs, n_points=512))

len(samples)

100%|███████████████████████████████████████████████████████| 21/21 [20:56<00:00, 59.85s/it]


662

# Upsample the low resolution dataset

## Load the upsampling model

In [44]:
s = torch.stack(samples).clone()
s.shape

torch.Size([662, 512, 3])

In [45]:
up_model = SPVUnet(point_channels=4, 
                   down_blocks = [[(64, 128, 192, 256, 256), (True, True, True, False), (None, None, None, None)]], # only one point skip connection during downsampling
                   up_blocks   = [[(256, 256, 192), (True, True), (None, None), (3, 3)], 
                                  [(192, 128, 64), (True, False), (None, None), (3, 3)]])
up_model.load_state_dict(torch.load('./checkpoints/SuperRes.pt')['state_dict'])

<All keys matched successfully>

In [46]:
up_model = up_model.cuda().eval()

## Upsample the dataset

In [47]:
up_sched = DDPMSparseCompletionSchedulerGPU(sigma='coef_bt')

samples = torch.stack(samples)
generated = []
for i in tqdm(range(0, 662, 32)):
    bs = min(662-i, 32)
    pc = samples[i:i+bs]
    generated.extend(up_sched.complete(pc, up_model, n_points=2048))

100%|███████████████████████████████████████████████████████| 21/21 [32:29<00:00, 92.85s/it]


# Evaluate the dataset

## Normalize the models and save ground_truths

In [49]:
path = "/home/tourloid/Desktop/PhD/Data/ShapeNetCore.v2.PC15k"
#path = "/home/vvrbeast/Desktop/Giannis/Data/ShapeNetCore.v2.PC15k"

test_loader = get_test_loader(path, ['chair'])

(1, 1, 1)
Total number of data:4612
Min number of points: (train)2048 (test)2048
(1, 1, 1)
Total number of data:662
Min number of points: (train)2048 (test)2048


In [52]:
generated.device

device(type='cpu')

In [63]:
#generated = torch.stack(generated)
all_sample = []
all_ref = []

for i, data in enumerate(test_loader):
    bs = min(662-i*32, 32)
    m, s = data['mean'].float(), data['std'].float()
    out_pc = generated[i:i+bs]
    te_pc = data['test_points']

    out_pc = out_pc * s + m
    te_pc = te_pc * s + m    

    all_sample.append(out_pc)
    all_ref.append(te_pc)

sample_pcs = torch.cat(all_sample, dim=0)
ref_pcs = torch.cat(all_ref, dim=0)

## Save the generated pcs 
Save to the *results* folder named as *generated_pcs.npy*

In [68]:
save_path='./results/'
np.save(os.path.join(save_path, 'generated_pcs.npy'), sample_pcs.cpu().numpy())
np.save(os.path.join(save_path, 'reference_pcs.npy'), ref_pcs.cpu().numpy())

## Evaluations

In [69]:
evaluate_gen(path, None, None, save_path='./results/', load_samples=True)

tensor(0.7296) tensor(0.7972)
tensor(-0.0038) tensor(0.0006)
Comparing 662 generated samples of shape [2048, 3] to 662 original samples of shape [2048, 3]


100%|█████████████████████████████████████████████████████| 662/662 [08:29<00:00,  1.30it/s]


{'1-NN-CD-acc': 0.9592145085334778,
 '1-NN-CD-acc_f': 0.9984894394874573,
 '1-NN-CD-acc_t': 0.9199395775794983,
 '1-NN-EMD-acc': 0.9728096723556519,
 '1-NN-EMD-acc_f': 0.9969788789749146,
 '1-NN-EMD-acc_t': 0.9486404657363892,
 'lgan_cov-CD': 0.06646525859832764,
 'lgan_cov-EMD': 0.07401812821626663,
 'lgan_mmd-CD': 0.004472191445529461,
 'lgan_mmd-EMD': 0.0240921787917614,
 'lgan_mmd_smp-CD': 0.002352555049583316,
 'lgan_mmd_smp-EMD': 0.017148727551102638}
'JSD: 0.08175276744320037'
